In [5]:
!pip3 install pymongo

     |████████████████████████████████| 378kB 2.7MB/s eta 0:00:01


In [119]:
!pip3 install folium

     |████████████████████████████████| 92kB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 133kB 3.3MB/s eta 0:00:01


In [120]:
from pymongo import MongoClient
import folium

In [104]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [105]:
db, coll = connectCollection('companies', 'companies')

In [106]:
print(coll.json)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies.json')


In [107]:
comp = list(coll.find({'$and': [{"offices.latitude": { '$nin': [None] }}, {"offices.longitude":{ '$nin': [None] }}, {"offices.city": "Amsterdam"}, {'deadpooled_year': None}]}))

In [117]:
#print(comp)

In [109]:
print(len(comp))
print(len(comp[1]['offices']))


45
3


In [111]:
print(len(comp))
comp[0]['offices']

45


[{'description': 'Arts & Crafts',
  'address1': 'Damrak 70, 254',
  'address2': '',
  'zip_code': '1012LM',
  'city': 'Amsterdam',
  'state_code': None,
  'country_code': 'NLD',
  'latitude': 52.3745228,
  'longitude': 4.8948623}]

In [112]:
print(len(comp[0]['offices']))
comp[0]['offices'][0]

1


{'description': 'Arts & Crafts',
 'address1': 'Damrak 70, 254',
 'address2': '',
 'zip_code': '1012LM',
 'city': 'Amsterdam',
 'state_code': None,
 'country_code': 'NLD',
 'latitude': 52.3745228,
 'longitude': 4.8948623}

In [113]:
def getLocation(companies):
    location =[]
    for i in range(len(companies['offices'])):
        longitude = companies['offices'][i]['longitude']
        latitude = companies['offices'][i]['latitude']
        loc = {
            'type':'Point',
            'coordinates':[longitude, latitude]
        }
        location.append(loc)
    return location

In [116]:
for cp in comp:
    value = {"$set": {'location':getLocation(cp)}}
    coll.update_one(cp, value)

In [154]:
comp[0]['location'][0]

{'type': 'Point', 'coordinates': [4.8948623, 52.3745228]}

In [151]:
comp[0]

{'_id': ObjectId('52cdef7c4bab8bd675297de6'),
 'name': 'Wakoopa',
 'permalink': 'wakoopa',
 'crunchbase_url': 'http://www.crunchbase.com/company/wakoopa',
 'homepage_url': 'http://www.wakoopa.com',
 'blog_url': 'http://blog.wakoopa.com',
 'blog_feed_url': 'http://blog.wakoopa.com/feed/',
 'twitter_username': 'wakoopa',
 'category_code': 'web',
 'number_of_employees': 8,
 'founded_year': 2007,
 'founded_month': 1,
 'founded_day': 1,
 'deadpooled_year': None,
 'deadpooled_month': None,
 'deadpooled_day': None,
 'deadpooled_url': None,
 'tag_list': 'socialnetwork, software, desktop, apps',
 'alias_list': '',
 'email_address': 'info@wakoopa.com',
 'phone_number': '',
 'description': 'Social network for desktop software',
 'created_at': 'Tue Feb 05 22:28:16 UTC 2008',
 'updated_at': 'Fri Sep 06 12:31:20 UTC 2013',
 'overview': '<p>Wakoopa creates digital DNA of today&#8217;s consumer. In a privacy conscious way, our technology tracks what websites are visited, what ads are seen, or what app

In [155]:
map_city=folium.Map(location=[52.37021,4.895168], zoom_start=12)
for cmp in comp:
    folium.Marker(cmp['location'][0]['coordinates'][::-1],
                radius=2,
                icon=folium.Icon(icon='home',color='red'), 
               ).add_to(map_city)
  

In [156]:
map_city